In [9]:
%cd /data/jupyter/root/ECNU/DataMining/L18-ImageObjectRecogDL/

/data/jupyter/root/ECNU/DataMining/L18-ImageObjectRecogDL


In [11]:
from __future__ import division
import os, sys

# 启动绘图
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw
import codecs
import csv
import warnings
warnings.filterwarnings('ignore')

import random
import time


# 手写数字识别

## 加载数据

In [12]:
class Data(object):
    def __init__(self):
        self._epochs_completed = 0
        self._index_in_epoch = 0
        
        train_x = []
        train_y = []
        
        image_dir =os.listdir('data/trainingDigits')
        for image_file_name in image_dir:
            image = []
            image_file_path = os.path.join('data/trainingDigits', image_file_name)
            fr = open(image_file_path)
            
            for line in fr.readlines():
                line = line.strip()
                list_line = []
                for i in range(32):
                    list_line.append(int(line[i]))
                image.append(list_line)
            train_x.append(np.asarray(image))
            label = np.zeros(10)
            label[int(image_file_name.split('_')[0])] = 10
            train_y.append(label)
        self.train_images = train_x
        self.train_labels = train_y
        # 混淆数据
        data = list(zip(self.train_images, self.train_labels))
        random.shuffle(data)
        self.train_images[:], self.train_labels[:] = zip(*data)
        
        self._num_examples = len(self.train_images)
        
    def batch_inter(self, batch_size):
        self._num_examples = len(self.train_images)
        start = self._index_in_epoch
        self._index_in_epoch += batch_size
        
        if self._index_in_epoch > self._num_examples:
            self._epochs_completed += 1
            data = list(zip(self.train_images, self.train_labels))
            random.shuffle(data)
            self.train_images[:], self.train_labels[:] = zip(*data)
            start = 0
            self._index_in_epoch = batch_size
            assert batch_sizea <= selflf._num_examples
        end = self._index_in_epoch
        return self.train_images[start:end], self.train_labels[start:end]
    
    def test_data(self):
        test_x = []
        test_y = []
        
        image_dir =os.listdir('data/testDigits')
        for image_file_name in image_dir:
            image = []
            image_file_path = os.path.join('data/testDigits', image_file_name)
            fr = open(image_file_path)
            for line in fr.readlines():
                line = line.strip()
                list_line = []
                for i in range(32):
                    list_line.append(int(line[i]))
                image.append(list_line)
            test_x.append(np.asarray(image))
            label = np.zeros(10)
            label[int(image_file_name.split('_')[0])] = 10
            test_y.append(label)
        self.test_images = test_x
        self.test_labels = test_y
        # 混淆数据
        data = list(zip(self.test_images, self.test_labels))
        random.shuffle(data)
        self.test_images[:], self.test_labels[:] = zip(*data)
        return self.test_images, self.test_labels
    
    def __len__(self):
        return self._num_examples
data = Data()

## 构建CNN模型

In [13]:
import tensorflow as tf
class Model_CNN_digit(object):
    def __init__(self):
        
        def weight_variable(shape):
            initial = tf.truncated_normal(shape, stddev=0.1)
            return tf.Variable(initial)
        def bias_variable(shape):
            initial = tf.constant(0.1, shape=shape)
            return tf.Variable(initial)
        # 卷积层
        def conv2d(x, W):
            return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
        # 池化层
        def max_pool_2x2(x):
            return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

        x = tf.placeholder(tf.float32, [None, 32, 32])
        y = tf.placeholder(tf.float32, [None, 10])
        x_image = tf.reshape(x, [-1, 32, 32, 1])

        # 第一个卷积层
        W_conv1 = weight_variable([5, 5, 1, 32])
        b_conv1 = bias_variable([32])
        # 使用conv2d 进行卷积
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
        h_pool1 = max_pool_2x2(h_conv1)

        # 第二个卷积层
        W_conv2 = weight_variable([5, 5, 1, 64])
        b_conv2 = bias_variable([64])
        # 使用conv2d 进行卷积
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        h_pool2 = max_pool_2x2(h_conv2)

        # 全连接层
        W_fcl = weight_variable([8 * 8 * 64, 1024])
        b_fcl = bias_variable([1024])
        h_pool2_flat = tf.reshape(h_pool2, [-1, 8 * 8* 64])
        h_fcl = tf.nn.relu(tf.matmul(h_pool2_flat, W_fcl) + b_fcl)

        # 减轻过拟合
        h_fcl_drop = tf.nn.dropout(h_fcl, keep_prob=0.75)

        W_fc2 = weight_variable([1024, 10])
        b_fc2 = bias_variable([10])
        pred = tf.nn.softmax(tf.matmul(h_fcl_drop, W_fc2) + b_fc2)

        cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

        correct_predicion = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_predicion, tf.float32))

        self.x = x
        self.y = y
        self.pred = pred
        self.cost = cost
        self.accuracy = accuracy
        self.optimizer = optimizer

    def train_model(self, sess, batch_xs, batch_ys):
        feed_dict ={
            self.x: batch_xs,
            self.y: batch_ys
        }
        to_return = {
            'train_op': self.optimizer,
            'loss': self.cost,
            'acc': self.accuracy,
        }
        return sess.run(to_return, feed_dict)


    def test_model(self, sess, batch_xs):
        feed_dict = {
            self.x: batch_xs
        }
        to_return = {
            'pred': self.pred
        }
        return sess.run(to_return, feed_dict)
    def eval_mode(self, sess, batch_xs, batch_ys):
        correct_predicion = tf.equal(tf.argmax(self.pred, 1), tf.argmax(self.y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_predicion, tf.float32))

        feed_dict ={
            self.x: batch_xs,
            self.y: batch_ys
        }
        to_return = {
            'acc': accuracy
        }
        return sess.run(to_return, feed_dict)



In [14]:
learning_rate = 1e-4
training_epochs = 50
batch_size = 100
display_step = 1
keep_prob = 0.75

In [15]:
model = Model_CNN_digit()
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [35]:
batch_xs, batch_ys = data.batch_inter(batch_size)
print(batch_xs[0])
print(batch_ys[0])
results = model.train_model(sess, batch_xs , batch_ys)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[ 0.  0.  0. 10.  0.  0.  0.  0.  0.  0.]


RuntimeError: Attempted to use a closed Session.

In [25]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cos = 0.
        avg_cos_list = []
        avg_acc = 0.
        total_batch = int(len(data) / batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = data.batch_inter(batch_size)
            print(batch_xs[0][0])
            print(batch_ys[0])
            results = model.train_model(sess, batch_xs , batch_ys)
            c = results['loss']
            ac = results['acc']
            
            avg_cos_list.append(c)
            
            avg_cos += c/total_batch
            avg_acc += ac/total_batch
        if (epoch + 1) % display_step == 0:
            print("CNN Epoch", '%04d'%(epoch+1), 
                  "cost=", "{:.9f}".format(avg_cos), 
                  "accuracy","{:.9f}".formato(avg_acc))
    print("finished")
    
    test_xs, test_ys = data.test_data()
    acc = model.eval_model(sess, test_xs, test_ys)
    print("Accuracy", acc)

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0.  0.  0. 10.  0.  0.  0.  0.  0.  0.]


InvalidArgumentError: input and filter must have the same depth: 32 vs 1
	 [[node Conv2D_3 (defined at /root/anaconda3/envs/DataMining/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'Conv2D_3':
  File "/root/anaconda3/envs/DataMining/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda3/envs/DataMining/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/root/anaconda3/envs/DataMining/lib/python3.7/site-packages/tornado/platform